Problem set 1b
Computational Linguistics
Fall 2023

This problem set involves working concretely with grammars, parsing, and the associated data structures in NLTK.


In [1]:
import nltk
from nltk.tree import Tree


1. In NLTK, create and display graphically Xbar trees in the style of Lecture 1
and Lecture Note 1 for the following three sentences.  Define the trees ta, tb,
and t3 and display them inline in your notebook.  In case of ambiguity, choose
the semantically plausible tree shape.

a. the title on every book under the table will have faded

b. she wondered whether he knew that they believed they admired Keisha

c. every eligeble candidate told a presiding supervisor about it 

In [2]:
t1 = Tree.fromstring("\
    (S (NP \
        (NP (DT the) (NN title))\
        (PP (IN on) (NP\
            (DT every)\
            (NP (NN book) (PP\
                (IN under) (NP (DT the) (NN table))))))) \
       (VP (MD will) (VP (VB have) (VBN faded) )))\
    ")

In [3]:
print(t1)
t1.draw()

(S
  (NP
    (NP (DT the) (NN title))
    (PP
      (IN on)
      (NP
        (DT every)
        (NP (NN book) (PP (IN under) (NP (DT the) (NN table)))))))
  (VP (MD will) (VP (VB have) (VBN faded))))


![t1](t1.png)

In [4]:
t2_s1 = Tree.fromstring("\
    (S (NP (PRP they))\
         (VP (VBD admired) (NP (NNP Keisha) )))")
t2_s2 = Tree.fromstring(f"\
    (S (NP (PRP they))\
         (VP (VBD believed) (SBAR {t2_s1} )))")
t2_s3 = Tree.fromstring(f"\
    (S (NP (PRP he))\
            (VP (VBD knew) (SBAR (IN that) {t2_s2} )))")
t2 = Tree.fromstring(f"\
    (S (NP (PRP She))\
            (VP (VBD wondered) (SBAR (IN whether) {t2_s3} )))")

In [5]:
print(t2)
t2.draw()

(S
  (NP (PRP She))
  (VP
    (VBD wondered)
    (SBAR
      (IN whether)
      (S
        (NP (PRP he))
        (VP
          (VBD knew)
          (SBAR
            (IN that)
            (S
              (NP (PRP they))
              (VP
                (VBD believed)
                (SBAR
                  (S
                    (NP (PRP they))
                    (VP (VBD admired) (NP (NNP Keisha)))))))))))))


![t2](t2.png)

In [6]:
np = Tree.fromstring("\
    (NP (DT every) (JJ eligible) (NN candidate))")
vp = Tree.fromstring("\
    (VP (VBD told) \
        (NP (DT a) (JJ presiding) (NN supervisor))\
        (PP (IN about) (NP (PRN it)) ))")
t3 = Tree.fromstring(f"\
    (S {np} {vp} )")

In [7]:
print(t3)
t3.draw()

(S
  (NP (DT every) (JJ eligible) (NN candidate))
  (VP
    (VBD told)
    (NP (DT a) (JJ presiding) (NN supervisor))
    (PP (IN about) (NP (PRN it)))))


![t3](t3.png)


2. Grammar of transitive verbs and relative clauses.

The following are transitive sentences.

the electrician repaired the router

the plumber damaged the sink

the plumber admired the electrician

In the following, what are called subject relative clauses have been added to one of the NPs.
This means that the relative pronoun "who" or "that" fills the subject position of
the embedded verb.

the electrician who admired the plumber repaired the router

the plumber admired the electrician who damaged the sink

the plumber repaired the sink that admired the router

In the following, what are called object relative clauses have been added to one of the NPs.
This means that the relative pronoun fills the object position of the embedded transitive verb.

the plumber who the electrician admires damaged the sink

the plumber damaged the sink that the electrician repaired.

More relative clauses can be added.  In the second example, there are two relative
on the noun "electrician".

the plumber who the electrician admires damaged the sink that the plumber repaired

the plumber admires the electrician who repaired the sink who the plumber admires


a. In NLTK, write a CFG in ***Chomsky normal form*** for sentences with this pattern. The terminal words should be the ones
that are seen above. Name the grammar g2.

Start with the following grammar, and add productions. In the symbols, the character 'a' indicates animate,
and the character 'i' indicates inanimate.

S -> NPa VP
S -> NPi VP
V -> 'admired' | 'envied'
V -> 'damaged' | 'repaired'
Na -> 'plumber' | 'electrician' 
Ni -> 'router' | 'sink' 
THE -> 'the'
NPa -> THE Na
NPi -> THE Ni
VP -> V NPa
VP -> V NPi
WHO -> 'who'
THAT -> 'that'


The grammar should impose the constraint that "electrician" and "plumber" go with the relative pronoun "who" and
not the relative pronoun "that", while "sink" and "router" go with the relative pronoun "that" and not the
relative pronoun "who". Use the distinction between 'a' and 'i' to impose this.
As came up in class, this contraint is arguably not really valid, but assume it.

*[NP the sink who the electrician damaged]
*[NP the electrician that damaged the sink]

Assume that semantically odd sentences like the following are ok.

the sink admires the router
the router repaired the plumber
the plumber repaired the electrician


In [9]:
x0 = """ S -> NPa VP
S -> NPi VP
V -> 'admired' | 'envied'
V -> 'damaged' | 'repaired'
Na -> 'plumber' | 'electrician' 
Ni -> 'router' | 'sink' 
THE -> 'the'
NPa -> THE Na
NPi -> THE Ni
VP -> V NPa
VP -> V NPi
WHO -> 'who'
THAT -> 'that'
"""

In [51]:
x2 = x0 + """
NPa -> NPa WHO VP
NPi -> NPi THAT VP

V -> 'admires' | 'envies'
V -> 'damages' | 'repairs'

NPa -> NPa WHO NP V
NPi -> NPi THAT NP V
"""
g2 = nltk.CFG.fromstring(x2)
print(g2)

Grammar with 25 productions (start state = S)
    S -> NPa VP
    S -> NPi VP
    V -> 'admired'
    V -> 'envied'
    V -> 'damaged'
    V -> 'repaired'
    Na -> 'plumber'
    Na -> 'electrician'
    Ni -> 'router'
    Ni -> 'sink'
    THE -> 'the'
    NPa -> THE Na
    NPi -> THE Ni
    VP -> V NPa
    VP -> V NPi
    WHO -> 'who'
    THAT -> 'that'
    NPa -> NPa WHO VP
    NPi -> NPi THAT VP
    V -> 'admires'
    V -> 'envies'
    V -> 'damages'
    V -> 'repairs'
    NPa -> NPa WHO NP V
    NPi -> NPi THAT NP V



3. Consider this sentence. Don't worry about the semantic implausibility.

the the router that repaired the sink that the plumber admired damaged the router that the plumber repaired

Using your grammar g2, define the following in Python.

 a list of strings sb that corresponds to the sentence above

 a parser p2 based on grammar g2

 a generator gb optained with p2 operating on sb

 a list of trees lb corresponding to gb (if you exhaust the generator gb in defining lb, redefine gb).
  The elements should be NLTK trees.

 the number nb of trees in lb


In [56]:
sent = "the sink that the electrician repaired"
s1 = sent.split()
parser = nltk.ChartParser(g2)
gen1 = parser.parse(s1)
for t in gen1: print(t)


4.  sent9.txt (supplied) is our list of sentences of lenght 9 that are in the
relative clause pattern.

Verify that your grammar g2 finds a tree for each of the sentences



5. Call a grammar symbol x nullable in grammar g if there some tree licensed by
g that has the terminal string [] (the empty list) and has the root label x.

a. In Stabler's grammar g1 (see g1.py, supplied), the symbol 'C' is nullable.
Show the tree that verifies nullability.

b. What are the nullable symbols in grammar gn (supplied)?
Show the trees that verify nullability.

c. Describe in words a definition of 'nullable in g' along these lines:

    symbol x is nullable in g iff there is a production x -> rhs in g such that ...

d. Define in python a function 'nullable' which when appled to a grammar in NLTK
format returns the set of nullable symbols.

Hint: start with the assumption that no symbols are nullable, then iteratively
search for nullable symbols by looking at the productions. A skeleton that does
something silly is supplied.

e. Test your function on some sample grammars including gn.